<a href="https://colab.research.google.com/github/sproboticworks/ml-course/blob/master/NLP%20Song%20Lyric%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/sproboticworks/master/assets/datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

# Tokenize Data

In [0]:
tokenizer = Tokenizer()

data = open('/tmp/irish-lyrics-eof.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print('Total Words: {}'.format(total_words))

# Prepare Data

In [0]:
input_sequences = []
for index, line in enumerate(corpus):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
    if index == 0:
        print("'"+line+"' => {}".format(token_list))
        print("Input Sequences :")
        print('\n'.join(map(str, input_sequences)))

In [0]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

first_line_len = len(corpus[0].split()) 
print("Padded Input Sequences for the first line :")
print(input_sequences[:first_line_len-1])

In [0]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [0]:
#Consider first sentence
print("Sentence : {}".format(corpus[0]))
print("Tokens : {}".format(input_sequences[first_line_len-2]))
print("X : {}".format(xs[first_line_len-2]))
print("Label : {}".format(labels[first_line_len-2]))
print("Y : {}".format(ys[first_line_len-2]))

#Build Model

In [0]:
embedding_dim = 100
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')                            
])

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.01), metrics=['accuracy'])

# Train Model

In [0]:
history = model.fit(xs, ys, epochs=100)

# Visualize Training Results

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [0]:
plot_graphs(history, 'accuracy')

#Generate Text

In [0]:
seed_text = "With great power comes great responsibility"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_labels = model.predict(token_list, verbose=0)
	predicted_index = np.argmax(predicted_labels, axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted_index:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)